<center><h2> Walmart Sales Forecasting </h2></center>

### Business objectives and constraints

1. The cost of a mis-classification can be very high.
2. There is some latency concerns.



```
# This is formatted as code
```

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot

from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import warnings

### Mounting with GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Reading the Data From the CSV file

- Stores.csv - It contains anonymized information on 45 stores, including their type and size. 
- Features.csv - It provides additional data pertaining to store, department, and regional activity for specific dates. 
- Train.csv - It consists of historical training data covering the period from 2010-02-05 to 2012-11-01.
 - Test.csv - It is identical to train.csv except that the weekly sales have been removed.

In [3]:
base_path = '/content/drive/MyDrive/walmart_prediction/data/'

train_df = pd.read_csv(f'{base_path}train.csv')
features_df = pd.read_csv(f'{base_path}features.csv')
stores_df = pd.read_csv(f'{base_path}stores.csv')
test_df = pd.read_csv(f'{base_path}test.csv')


In [4]:
train_df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [6]:
train_df.describe()

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


In [7]:
train_df.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [8]:
features_df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [9]:
features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [10]:
features_df.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


In [11]:
features_df.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [12]:
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [13]:
stores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [14]:
stores_df.describe()

,Store,Size
count,45.000000,45.000000
mean,23.000000,130287.600000
std,13.133926,63825.271991
min,1.000000,34875.000000
25%,12.000000,70713.000000
50%,23.000000,126512.000000
75%,34.000000,202307.000000
max,45.000000,219622.000000


In [15]:
stores_df.isnull().sum()

Store    0
Type     0
Size     0
dtype: int64

Visualizing the Type of the Stores along with their percentage

In [16]:
type_counts = stores_df["Type"].value_counts()
top_types = type_counts[:10].index
top_values = type_counts[:10].values
colors = stores_df["Type"]

fig = go.Figure(data=[go.Pie(labels=top_types, values=top_values, textinfo="label+percent", marker=dict(colors=colors))])

fig.show()


Type A has the max number of store. Since features and stores have 'Store' as a common column lets merge the data



In [17]:
# Using the merge function to merge ans we are merging along the common column named Store
dataset = features_df.merge(stores_df, how='inner', on='Store')
dataset.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315


In [18]:
stores_df.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [19]:
# Getting the info
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8190 entries, 0 to 8189
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
 12  Type          8190 non-null   object 
 13  Size          8190 non-null   int64  
dtypes: bool(1), float64(9), int64(2), object(2)
memory usage: 903.8+ KB


since the Date in the above dataset is string value we can convert them into datetime using the datetime

In [20]:
# Describing the dataset
dataset.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000,8190.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821,130287.600000
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259,63115.971429
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000,34875.000000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000,70713.000000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000,126512.000000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000,202307.000000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000,219622.000000


Here Date is of string type

In [21]:
# Getting the info
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Store      115064 non-null  int64 
 1   Dept       115064 non-null  int64 
 2   Date       115064 non-null  object
 3   IsHoliday  115064 non-null  bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 2.7+ MB


Visualizing the top 10 Department in the train dataset along with their percentage

In [22]:
top_10_depts = train_df["Dept"].value_counts().head(10)
labels = top_10_depts.index #working with top10 index
values = top_10_depts.values #working with top10 values
colors = train_df["Dept"]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, textinfo="label+percent", marker=dict(colors=colors))])
fig.show()

Visualizing the IsHoliday Data

In [ ]:
holiday_labels = train_df['IsHoliday'].value_counts().index
holiday_counts = train_df['IsHoliday'].value_counts().values

holiday_colors = train_df['IsHoliday']
fig = go.Figure(data=[go.Pie(labels=holiday_labels, values=holiday_counts, textinfo="label+percent", marker=dict(colors=holiday_colors))])
fig.show()

As per the plot for the 93% of the time there is no Holiday

In [ ]:
# To know more about the test_df
test_df.describe()

In [ ]:
train_df.info()

In [ ]:
from datetime import datetime
#conveting the string format of date in dataset to datetime format
dataset['Date'] = pd.to_datetime(dataset['Date'])
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df['Date'] = pd.to_datetime(test_df['Date'])

In [ ]:
train_df.info()

In [ ]:
dataset.info()

In [ ]:
dataset['Week'] = dataset.Date.dt.isocalendar().week # for the week data
dataset['Year'] = dataset.Date.dt.isocalendar().year # for the year data

In [ ]:
dataset.head()

In [ ]:
# Merging with train_df

train_merge = train_df.merge(dataset, how='inner', on=['Store', 'Date', 'IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
# Merging with test_df

test_merge = test_df.merge(dataset, how='inner', on=['Store', 'Date', 'IsHoliday']).sort_values(by=['Store','Dept','Date']).reset_index(drop=True)

In [ ]:
#Creating a reusable scatter plot function which plots the given data points
def scatter_plot(train_merge, column):
    plt.figure()
    plt.scatter(train_merge[column], train_merge['Weekly_Sales'])
    plt.ylabel('Weekly_Sales')
    plt.xlabel(column)


In [ ]:
columns = ['Fuel_Price', 'Size', 'CPI', 'Type', 'IsHoliday', 'Unemployment', 'Temperature', 'Store', 'Dept']
for column in columns:
    scatter_plot(train_merge, column)

Let's have a look at the Average Weekly Sales per Year and find out if there is any other holiday peak sales that were not considered by 'IsHoliday' filed

In [ ]:

#Creating a line plot using seaborn to see whether he sales have peaked during the holidays.
week_2010 = train_merge[train_merge['Year'] == 2010]
avg_weekly_sales_2010 = week_2010.groupby('Week')['Weekly_Sales'].mean()
avg_weekly_sales_2010 = np.array(avg_weekly_sales_2010, dtype=float)
ax = sns.lineplot(avg_weekly_sales_2010)

ax.set_ylabel('Weekly_Sales')
ax.set_xlabel('Weeks')

plt.title('Average Weekly Sales Per Year', fontsize=16)
plt.show()


In [ ]:
# Average Weekly Sales for the year 2011
week_2011 = train_merge[train_merge['Year'] == 2011]
avg_weekly_sales_2011 = week_2011.groupby('Week')['Weekly_Sales'].mean()
avg_weekly_sales_2011 = np.array(avg_weekly_sales_2011, dtype=float)

ax = sns.lineplot(avg_weekly_sales_2011)
ax.set_ylabel('Weekly_Sales')
ax.set_xlabel('Weeks')

plt.title('Average Weekly Sales Per Year', fontsize=16)
plt.show()


In [ ]:
# Average Weekly Sales for the year 2012
week_2012 = train_merge[train_merge['Year'] == 2012]
avg_weekly_sales_2012 = week_2012.groupby('Week')['Weekly_Sales'].mean()
avg_weekly_sales_2012 = np.array(avg_weekly_sales_2012, dtype=float)

ax = sns.lineplot(avg_weekly_sales_2012)
ax.set_ylabel('Weekly_Sales')
ax.set_xlabel('Weeks')

plt.title('Average Weekly Sales Per Year', fontsize=16)
plt.show()


In [ ]:
# Plotting the above three plot together 
plt.figure(figsize=(20,8))
sns.lineplot(avg_weekly_sales_2010, label='2010')
sns.lineplot(avg_weekly_sales_2011, label='2011')
sns.lineplot(avg_weekly_sales_2012, label='2012')
plt.grid()
plt.xticks(np.arange(1,60, step=1))
plt.legend(loc='best', fontsize=16)
plt.title('Average Weekly Sales Per Year', fontsize=16)
plt.ylabel('Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
plt.show()


In [ ]:
# Calculate the mean weekly sales per department
mean_sales = train_merge.groupby(['Dept'])['Weekly_Sales'].mean()

# Plot the mean sales per department as a bar plot
plt.figure(figsize=(25,12))
sns.barplot(x=mean_sales.index, y=mean_sales.values, palette='dark')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.title('Average Sales per Department', fontsize=20)
plt.xlabel('Department', fontsize=16)
plt.ylabel('Sales', fontsize=16)
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Calculate the mean weekly sales per store
mean_sales = train_merge.groupby(['Store'])['Weekly_Sales'].mean()

# Plot the mean sales per store as a bar plot
plt.figure(figsize=(20,12))
sns.barplot(x=mean_sales.index, y=mean_sales.values, palette='dark')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.title('Average Sales per Store', fontsize=20)
plt.xlabel('Store', fontsize=16)
plt.ylabel('Sales', fontsize=16)
plt.xticks(rotation=90)
plt.show()

In [ ]:
#Creating a correlation matrix to check how the variables are connected to each other.

sns.set(style="white")
correlation = train_merge.corr()
mask = np.triu(np.ones_like(correlation, dtype=bool))

fig, ax = plt.subplots(figsize=(20, 15))
palette = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(correlation, mask=mask, cmap=palette, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

plt.title("Correlation Matrix", fontsize=18)
plt.show()


In [ ]:
# Dropping down the variables that have weak correlation

train_merge = train_merge.drop(columns=['Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])
test_merge = test_merge.drop(columns=['Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'])

In [ ]:
train_merge.head()

In [ ]:
test_merge.head()

In [ ]:
# Counting the number of unique values
train_df['Dept'].nunique()

In [ ]:
# Counting the number of unique values
train_df['Store'].nunique()

In [ ]:
# Getting the columns in the train_merge 
train_merge.columns

In [ ]:
# Getting the columns in test_merge
test_merge.columns

**Dividing the data into Train and Test**

In [ ]:
X = train_merge[['Store','Dept','IsHoliday','Size','Week','Year']] # Selecting the values to be taken in the X
y = train_merge['Weekly_Sales'] 

**Model Building**

In [ ]:
# Splitting the data into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
X_train.info()

In [ ]:
# Performing GridSearchCV on Ridge Regression

params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
ridge = Ridge()
ridge_regressor = GridSearchCV(ridge, params, cv=7, scoring='neg_mean_absolute_error', n_jobs=-1)
ridge_regressor.fit(X_train, y_train)

In [ ]:
# Predicting train and test results
y_train_pred = ridge_regressor.predict(X_train)
y_test_pred = ridge_regressor.predict(X_test)

In [ ]:
def compare_results(prediction, actual):
    fig, ax = plt.subplots()
    ax.scatter(prediction, actual)

    # Adding labels and title to the plot
    # Add x-axis label
    ax.set_xlabel("X-axis Label")
    # Add y-axis label
    ax.set_ylabel("Y-axis Label")

    plt.title("Prediction vs Actual Results")
    plt.plot([0, 600000], [0, 600000], color='red')
    # Show the plot
    plt.show()

***Evaluation metrics for Ridge Regression***

In [ ]:
print("Train Results for Ridge Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Ridge Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_test.values, y_test_pred)))
print("R-Squared: ", r2_score(y_test.values, y_test_pred))

In [ ]:
# Performing GridSearchCV on Lasso Regression

params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
lasso = Lasso()
lasso_regressor = GridSearchCV(lasso, params, cv=15, scoring='neg_mean_absolute_error', n_jobs=-1)
lasso_regressor.fit(X_train, y_train)


In [ ]:
# Predicting train and test results
y_train_pred = lasso_regressor.predict(X_train)
y_test_pred = lasso_regressor.predict(X_test)

***Evaluation metrics for Lasso Regression***

In [ ]:
print("Train Results for Lasso Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Lasso Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_test, y_test_pred)))
print("R-Squared: ", r2_score(y_test, y_test_pred))

In [ ]:
# Performing GridSearchCV on Decision Tree Regression
from sklearn.tree import DecisionTreeRegressor

depth = list(range(3,30))
param_grid = dict(max_depth = depth)
tree_regressor = GridSearchCV(DecisionTreeRegressor(), param_grid, cv = 10)
tree_regressor.fit(X_train,y_train)


In [ ]:
# Predicting train and test results
y_train_pred = tree_regressor.predict(X_train)
y_test_pred = tree_regressor.predict(X_test)

In [ ]:
#The data points that fell directly on the line are perfectly predicted and the points above the 
# line passing through origin has been overestimated and points below the line are underestimated.
compare_results(y_test_pred, y_test)

***Evaluation Metrics for Tree Regression***

In [ ]:
print("Train Results for Decision Tree Regressor Model:")
print("Root Mean squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Decision Tree Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_test, y_test_pred)))
print("R-Squared: ", r2_score(y_test, y_test_pred))

In [ ]:
# Performing RandomsearchCV on Random Forest Regression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'n_estimators': [100, 200],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_regressor = RandomForestRegressor()
random_search = RandomizedSearchCV(
    rf_regressor, param_grid, n_iter=3,
    scoring='neg_mean_absolute_error', cv=3, n_jobs=-1
)
random_search.fit(X_train, y_train)


In [ ]:
# Predicting train and test results
y_train_pred = random_search.predict(X_train)
y_test_pred = random_search.predict(X_test)

In [ ]:
compare_results(y_test_pred, y_test)

***Evaluation metrics for Random Forest regression model***

In [ ]:
print("Train Results for Random Forest Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-Squared: ", r2_score(y_train.values, y_train_pred))

In [ ]:
print("Test Results for Random Forest Regressor Model:")
print("Root Mean Squared Error: ", sqrt(mse(y_test, y_test_pred)))
print("R-Squared: ", r2_score(y_test, y_test_pred))

**Conclusion**
With the above implementation it was possible to forecast the prediction for the Walmart Stores